## Overview

The Gemini File API provides a simple way for developers to upload files and use them with the Gemini API in multimodal scenarios. This notebook shows how to use the File API to upload an image and include it in a `GenerateContent` call to the Gemini API. For more information, refer to the [File API FAQ](https://docs.google.com/document/d/1WBVc5W6PZvgaHLV43UGSrtwHqUmofPT0K0oHuNd7GHA) or the [API documentation](https://ai.google.dev/api/rest/v1beta/files).

Note: This API is currently in beta and is [only available in certain regions](https://ai.google.dev/available_regions).

## Setup


### Install the Gemini API Python SDK



In [ ]:
!pip install -U -q google.generativeai # Install the Python SDK

### Authentication Overview

**Important:** The File API uses API keys for authentication and access. Uploaded files are associated with the API key's cloud project. Unlike other Gemini APIs that use API keys, your API key also grants access data you've uploaded to the File API, so take extra care in keeping your API key secure. For best practices on securing API keys, refer to Google's [documentation](https://support.google.com/googleapi/answer/6310037).

### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](https://github.com/google-gemini/gemini-api-cookbook/blob/main/quickstarts/Authentication.ipynb) for an example.

In [ ]:
import google.generativeai as genai
from IPython.display import Markdown
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## Extract Video Image Frames

The Gemini API currently does not support video files directly. Instead, you can provide a series of timestamps and image files. In this quickstart, we will extract 1 frame a second from a the short film "Big Buck Bunny" file using [OpenCV](https://docs.opencv.org/4.x/d6/d00/tutorial_py_root.html).

> "Big Buck Bunny" is (c) copyright 2008, Blender Foundation / www.bigbuckbunny.org and [licensed](https://peach.blender.org/about/) under the [Creative Commons Attribution 3.0](http://creativecommons.org/licenses/by/3.0/) License.

In [ ]:
video_file_name = "https://download.blender.org/peach/bigbuckbunny_movies/BigBuckBunny_320x180.mp4"

To upload your own video, see the [Appendix section](#scrollTo=OexqR86FIiCu).

### Extract frames using OpenCV

The following code uses OpenCV to extract image frames from the video at 1 frame per second.

In [ ]:
import cv2
import os
import shutil

# Create or cleanup existing extracted image frames directory.
FRAME_EXTRACTION_DIRECTORY = "/content/frames"
FRAME_PREFIX = "_frame"
def create_frame_output_dir(output_dir):
  if not os.path.exists(output_dir):
    os.makedirs(output_dir)
  else:
    shutil.rmtree(output_dir)
    os.makedirs(output_dir)

def extract_frame_from_video(video_file_path):
    print(f"Extracting {video_file_path} at 1 frame per second. This might take a bit...")
    create_frame_output_dir(FRAME_EXTRACTION_DIRECTORY)
    vidcap = cv2.VideoCapture(video_file_path)
    fps = vidcap.get(cv2.CAP_PROP_FPS)
    frame_duration = 1 / fps  # Time interval between frames (in seconds)
    output_file_prefix = os.path.basename(video_file_path).replace('.', '_')
    frame_count = 0
    count = 0
    while vidcap.isOpened():
        success, frame = vidcap.read()
        if not success: # End of video
            break
        if int(count / fps) == frame_count: # Extract a frame every second
            min = frame_count // 60
            sec = frame_count % 60
            time_string = f"{min:02d}:{sec:02d}"
            image_name = f"{output_file_prefix}{FRAME_PREFIX}{time_string}.jpg"
            output_filename = os.path.join(FRAME_EXTRACTION_DIRECTORY, image_name)
            cv2.imwrite(output_filename, frame)
            frame_count += 1
        count += 1
    vidcap.release()
    print(f"Completed video frame extraction!\n\nExtracted: {frame_count} frames")

extract_frame_from_video(video_file_name)

## Upload a file to the File API

The File API lets you upload a variety of multimodal MIME types, including images and audio formats. The File API handles inputs that can be used to generate content with [`model.generateContent`](https://ai.google.dev/api/rest/v1/models/generateContent) or [`model.streamGenerateContent`](https://ai.google.dev/api/rest/v1/models/streamGenerateContent).

The File API accepts files under 2GB in size and can store up to 20GB of files per project. Files last for 2 days and cannot be downloaded from the API.

In [ ]:
import os

class File:
  def __init__(self, file_path: str, display_name: str = None):
    self.file_path = file_path
    if display_name:
      self.display_name = display_name
    self.timestamp = get_timestamp(file_path)

  def set_file_response(self, response):
    self.response = response

def get_timestamp(filename):
  """Extracts the frame count (as an integer) from a filename with the format
     'output_file_prefix_frame00:00.jpg'.
  """
  parts = filename.split(FRAME_PREFIX)
  if len(parts) != 2:
      return None  # Indicates the filename might be incorrectly formatted
  return parts[1].split('.')[0]

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)  # Sort alphabetically
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API
# Only upload a 10 second slice of files to reduce upload time.
# Change full_video to True to upload the whole video.
full_video = False

uploaded_files = []
print(f'Uploading {len(files_to_upload) if full_video else 10} files. This might take a bit...')

for file in files_to_upload if full_video else files_to_upload[40:50]:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)

print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

## List Files

After uploading the file, you can verify the API has successfully received the files by calling `files.list`.

`files.list` lets you see all files that have been uploaded to the File API that are associated with the Cloud project your API key belongs to. Only the `name` (and by extension, the `uri`) are unique. Only use the `displayName` to identify files if you manage uniqueness yourself.

In [ ]:
# List files uploaded in the API
for n, f in zip(range(len(uploaded_files)), genai.list_files()):
  print(f.uri)

## Generate Content

After the file has been uploaded, you can make `GenerateContent` requests that reference the File API URI.

To understand videos with Gemini 1.5 Pro, provide 2 consecutive `Part`s for each frame: a `text` part with the **timestamp** and `fileData` part with the frame's **image URI**:

```
part { text = "00:00" }
part { fileData = fileData {
  fileUri = "https://generativelanguage.googleapis.com/v1/files/frame-0"
  mimeType = "image/jpeg"
}}
```

In [ ]:
# Create the prompt.
prompt = "Describe this video."

# Set the model to Gemini 1.5 Pro.
model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest")

# Make GenerateContent request with the structure described above.
def makeRequest(prompt, files):
  request = [prompt]
  for file in files:
    request.append(file.timestamp)
    request.append(file.response)
  return request

# Make the LLM request.
request = makeRequest(prompt, uploaded_files)
response = model.generate_content(request,
                                  request_options={"timeout": 600})
print(response.text)

If you `print(request)`, you'd see the request follows the structure for video understanding in Gemini 1.5 Pro:

```python
['Describe this video.', '00:40', <google.generativeai.types.file_types.File object at 0x7d9363a88760>, '00:41', <google.generativeai.types.file_types.File object at 0x7d9363ca7df0>, '00:42', <google.generativeai.types.file_types.File object at 0x7d9363a8a410>, '00:43', <google.generativeai.types.file_types.File object at 0x7d9363a89510>, '00:44', <google.generativeai.types.file_types.File object at 0x7d9363a88d00>, '00:45', <google.generativeai.types.file_types.File object at 0x7d9363a8abf0>, '00:46', <google.generativeai.types.file_types.File object at 0x7d9363a8b130>, '00:47', <google.generativeai.types.file_types.File object at 0x7d9363a88d30>, '00:48', <google.generativeai.types.file_types.File object at 0x7d9363a8a920>, '00:49', <google.generativeai.types.file_types.File object at 0x7d9363aa00d0>]
```



## Delete Files

Files are automatically deleted after 2 days or you can manually delete them using `files.delete()`.

In [ ]:
print(f'Deleting {len(uploaded_files)} images. This might take a bit...')
for file in uploaded_files[1:]:
  print(file.response.name)
  genai.delete_file(file.response.name)
  print(f'Deleted {file.file_path} at URI {file.response.uri}')
print(f"Completed deleting files!\n\nDeleted: {len(uploaded_files)} files")

## Appendix: Uploading files to Colab
<a name="uploading_files_to_colab"></a>
This notebook uses the File API with files that were downloaded from the internet. If you're running this in Colab and want to use your own files, you first need to upload them to the colab instance.

First, click **Files** on the left sidebar, then click the **Upload** button:

<img width=400 src="https://ai.google.dev/tutorials/images/colab_upload.png">

Next, we'll upload that file to the File API. In the form for the code cell below, enter the filename for the file you uploaded and provide an appropriate display name for the file, then run the cell.

In [ ]:
# Specify the file to upload
my_filename = "sample_video.mp4" # @param {type:"string"}
my_file_display_name = "Sample Video" # @param {type:"string"}

# Extract frames
extract_frame_from_video(my_filename)

# Process each frame in the output directory
files = os.listdir(FRAME_EXTRACTION_DIRECTORY)
files = sorted(files)  # Sort alphabetically
files_to_upload = []
for file in files:
  files_to_upload.append(
      File(file_path=os.path.join(FRAME_EXTRACTION_DIRECTORY, file)))

# Upload the files to the API.
uploaded_files = []
print(f'Uploading {len(files_to_upload)} files. This might take a bit...')
for file in files_to_upload:
  print(f'Uploading: {file.file_path}...')
  response = genai.upload_file(path=file.file_path)
  file.set_file_response(response)
  uploaded_files.append(file)
print(f"Completed file uploads!\n\nUploaded: {len(uploaded_files)} files")

# Make the LLM request.
prompt = "Tell me about this video."
response = model.generate_content(makeRequest(prompt, uploaded_files))
Markdown(">" + response.text)